In [1]:

import yaml
import cv2
import numpy as np
import glob
import os
import sys
import matplotlib.pyplot as plt
import open3d as o3d
import time
import math
sys.path.append("src/")
from src.world import CarlaWorld
from src.ego_vehicle import EgoVehicle

try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass
import carla

try:
    import pygame
    from pygame.locals import K_ESCAPE
    from pygame.locals import K_q
except ImportError:
    raise RuntimeError('cannot import pygame, make sure pygame package is installed')

from src.utils import capture_data
from src.pygame_display import DisplayManager


with open('cfg\\vehicle_cfg.yaml', 'r') as f:
    vehicle_cfg = yaml.safe_load(f) 
    
with open('cfg\\config.yaml', 'r') as f:
    cfg = yaml.safe_load(f) 
    
carla_world = CarlaWorld(cfg)

bp_lib = carla_world.world.get_blueprint_library()
ego_vehicle = EgoVehicle(bp_lib, vehicle_cfg)
ego_vehicle.spwan_ego_vehicle(carla_world.world)
display_man = None
if cfg['sensor_preview']:
    grid_size = [math.ceil(ego_vehicle.num_cameras/3), 3]
    display_man = DisplayManager(grid_size, window_size=[1280, 720])

ego_vehicle.sensor_setup(carla_world.world, display_man, enable_lidar_vis=cfg['sensor_preview'])
static_bboxes = carla_world.world.get_level_bbs(carla.CityObjectLabel.Car)
carla_world.spawn_actors()
ego_vehicle.ego_vehicle.set_autopilot(True)

carla_world.set_synchronous()

out_dir = cfg['out_dir']
capture_frequency = cfg['capture_frequency']
simulation_frequency = cfg['fps']
delta_tick = int(simulation_frequency/capture_frequency)
assert delta_tick > 0, "please reduce capture_frequency"
frame_no = 0
call_exit = False
time.sleep(1.0)
rgb, depth, sem_seg, point_cloud = None, None, None, None
# Main loop
while True:
    frame_id = carla_world.tick()    
    vehicles = carla_world.world.get_actors().filter('vehicle.*')

    # Data Capture
    if cfg['capture_data']:
        for sensor in ego_vehicle.sensors:
            
            if sensor.sensor_type == 'RGBCamera':
                rgb, depth, sem_seg = sensor.retrive_data(frame_id, 2.0)
                if frame_no % delta_tick == 0:
                    capture_data(frame_no=frame_no, out_dir=cfg['out_dir'], sensor_name=sensor.sensor_name, rgb=rgb, depth=depth, semantic_mask=sem_seg)
                    print(f"Saved Camera Frame no {frame_no} for {sensor.sensor_name}")

            if sensor.sensor_type == 'LiDAR':
                point_cloud, bb = sensor.retrive_data(frame_id, 2.0)        
                if frame_no % delta_tick == 0:
                    capture_data(frame_no=frame_no, out_dir=cfg['out_dir'], sensor_name=sensor.sensor_name, lidar_pc=point_cloud)
                    print(f"Saved Lidar Frame no {frame_no} for {sensor.sensor_name}")
                    
        frame_no += 1
    
    # Visualization
    if cfg['sensor_preview']:         
        pygame.display.flip()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                call_exit = True
            elif event.type == pygame.KEYDOWN:
                if event.key == K_ESCAPE or event.key == K_q:
                    call_exit = True
                    break

        if call_exit:
            pygame.display.quit()
            for sensor in ego_vehicle.sensors:
                if sensor.sensor_type == 'LiDAR' and sensor.vis:
                    sensor.vis.destroy_window()
                    
            break


carla_world.restore()
carla_world.destroy_actors()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
pygame 2.1.2 (SDL 2.0.18, Python 3.7.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
rgb time 546.6231268234551
depth time 546.6231268234551
sem time 546.6231268234551
rgb time 546.6231268234551
rgb time 546.6231268234551
[Open3D WARNING] Write PCD failed: unable to generate header.
rgb time 546.7231268249452
depth time 546.7231268249452
sem time 546.7231268249452
rgb time 546.7231268249452
rgb time 546.7231268249452
rgb time 546.8231268264353
depth time 546.8231268264353
sem time 546.8231268264353
rgb time 546.8231268264353
rgb time 546.8231268264353
rgb time 546.9231268279254
depth time 546.9231268279254
sem time 546.9231268279254
rgb time 546.9231268279254
rgb time 546.9231268279254
rgb time 547.0231268294156
depth time 547.0231268294156
sem time 547.0231268294156
rgb time 547.0231268294156

: 

: 

In [2]:
carla_world.restore()
carla_world.destroy_actors()
pygame.display.quit()
for sensor in ego_vehicle.sensors:
    if sensor.sensor_type == 'LiDAR' and sensor.vis:
        sensor.vis.destroy_window()

In [ ]:
num_cameras = 6
import math
grid_size = [math.ceil(num_cameras/3), 3]
grid_size

In [ ]:
carla_world.world.get_actor(carla.CityObjectLabel.Car)

In [ ]:
import cv2
import matplotlib.pyplot as plt
from pygame_display import DisplayManager
import pygame
from pygame.locals import K_ESCAPE
from pygame.locals import K_q

img = cv2.imread(r"D:\rbe\lara\renders\untitled1.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
display = DisplayManager([1,3], [1280, 720])
call_exit = False
while True:
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            call_exit = True
        elif event.type == pygame.KEYDOWN:
            if event.key == K_ESCAPE or event.key == K_q:
                call_exit = True
                break
    if call_exit:
        break
    
    resize = display.get_display_size()
    img = cv2.resize(img, resize)
    surface = pygame.surfarray.make_surface(img)
    offset = display.get_display_offset([0,3])
    display.display.blit(surface, offset)

plt.imshow(img)

In [ ]:
import numpy as np
r = 1
degs = [0, 60, 120, 180, 240, 300]

for deg in degs:
    x = np.cos(np.deg2rad(deg))
    y = np.sin(np.deg2rad(deg))
    print(x,y)

In [ ]:
a = 4

if a is not None:
    print(a)